In [2]:
suppressMessages(library(rwwa))
png_res <- 240

# Check results using original functions

In [23]:
nsamp <- 1000

varnm <- "rx1day-ondjfm"
rnm <- "s"

# load covariates
gmst <- read.table("ts-obs/gmst.dat", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2026]
nao <- load_ts("ts-obs/med-storms_nao-djf_era5-stn.dat", col.names = c("year", "nao"))

cov_df <- merge(gmst, nao, all.x = T)
cov_df$gmst <- cov_df$gmst - cov_df$gmst[cov_df$year == 2026]

# define factual & counterfactual climates
cov_2026 <- cov_df[cov_df$year == 2026,c(-1),drop = F]
cov_cf <- rbind("pi" = cov_2026 - c(1.3,0),
                "naoneutral" = c(cov_2026$gmst, 0),
                "pineutral" = c(-1.3, 0))

era5 <- merge(cov_df, load_ts(paste0("ts-obs/med-storms_",varnm,"_",rnm,"_era5.dat"), col.names = c("year", "x"))) # 1950-2026
eobs <- merge(cov_df, load_ts(paste0("ts-obs/med-storms_",varnm,"_",rnm,"_eobs.dat"), col.names = c("year", "x"))) # 1920-2026

mdl_era5_gmstnao <- fit_ns("gev", "fixeddisp", data = era5, covnm = c("gmst", "nao"), varnm = "x", lower = F)
mdl_era5_gmst <- fit_ns("gev", "fixeddisp", data = era5, covnm = c("gmst"), varnm = "x", lower = F)

mdl_eobs_gmstnao <- fit_ns("gev", "fixeddisp", data = eobs[!is.na(eobs$na),], covnm = c("gmst", "nao"), varnm = "x", lower = F)
mdl_eobs_gmst1920 <- fit_ns("gev", "fixeddisp", data = eobs, covnm = c("gmst"), varnm = "x", lower = F)
mdl_eobs_gmst1950 <- fit_ns("gev", "fixeddisp", data = eobs[!is.na(eobs$na),], covnm = c("gmst"), varnm = "x", lower = F)

In [24]:
mlist <- list("ERA5 - GMST only" = mdl_era5_gmst,
            "ERA5 - GMST + NAO" = mdl_era5_gmstnao,
            "EObs - GMST only (1920-2026)" = mdl_eobs_gmst1920,
            "EObs - GMST only (1950-2026)" = mdl_eobs_gmst1950,
            "EObs - GMST + NAO" = mdl_eobs_gmstnao)

res <- rbind.fill(sapply(mlist, function(mdl) data.frame(t(mdl_ests(mdl, cov_f = cov_2026, cov_cf = cov_cf))), simplify = F))
rownames(res) <- names(mlist)

In [25]:
res[,c("return_period","PR_pi","dI_rel_pi")]
# E-Obs results fairly stable regardless of whether using 1920 or 1950 onwards

,return_period,PR_pi,dI_rel_pi
,<dbl>,<dbl>,<dbl>
ERA5 - GMST only,82.613075,52.789138,17.65778
ERA5 - GMST + NAO,39.799635,64.942525,23.00887
EObs - GMST only (1920-2026),2.998259,6.873176,38.15117
EObs - GMST only (1950-2026),2.734766,7.319387,42.38718
EObs - GMST + NAO,2.223773,8.818264,49.21544


## Get results for RP40 event

In [26]:
mlist_rp40 <- lapply(mlist, function(mdl) {
    rl40 <- eff_return_level(mdl, rp = 40, fixed_cov = cov_2026)
    fit_ns(dist = mdl$dist, type = mdl$type, data = mdl$data, varnm = mdl$varnm, covnm = mdl$covnm, lower = mdl$lower, ev = rl40, ev_year = 2026)
})

In [27]:
res40 <- rbind.fill(sapply(mlist_rp40, function(mdl) data.frame(t(mdl_ests(mdl, cov_f = cov_2026, cov_cf = cov_cf))), simplify = F))
rownames(res40) <- names(mlist_rp40)

In [28]:
res[,c("return_period","PR_pi","dI_rel_pi")]        # print the original results
res40[,c("return_period","PR_pi","dI_rel_pi")]
# E-Obs results fairly stable regardless of whether using 1920 or 1950 onwards

,return_period,PR_pi,dI_rel_pi
,<dbl>,<dbl>,<dbl>
ERA5 - GMST only,82.613075,52.789138,17.65778
ERA5 - GMST + NAO,39.799635,64.942525,23.00887
EObs - GMST only (1920-2026),2.998259,6.873176,38.15117
EObs - GMST only (1950-2026),2.734766,7.319387,42.38718
EObs - GMST + NAO,2.223773,8.818264,49.21544


,return_period,PR_pi,dI_rel_pi
,<dbl>,<dbl>,<dbl>
ERA5 - GMST only,40,21.57370,17.65778
ERA5 - GMST + NAO,40,65.42776,23.00887
EObs - GMST only (1920-2026),40,26.32044,38.15117
EObs - GMST only (1950-2026),40,29.46276,42.38718
EObs - GMST + NAO,40,48.41232,49.21544


In [ ]:
boot_all <- lapply(mlist_rp40, boot_ci, cov_f = cov_2026, cov_cf = cov_cf, nsamp = nsamp)

In [39]:
res_obs <- rbind.fill(lapply(boot_all, function(res) data.frame(t(unlist(lapply(rownames(res), function(cnm) setNames(res[cnm,], paste(gsub("_", "-", cnm), c("est", "lower", "upper"), sep = "_"))))))))
rownames(res_obs) <- names(boot_all)            

In [41]:
write.csv(res_obs, paste0("res-obs_",varnm,"_",rnm,"_rp40.csv"))

In [42]:
res_obs

,mu0_est,mu0_lower,mu0_upper,sigma0_est,sigma0_lower,sigma0_upper,alpha.gmst_est,alpha.gmst_lower,alpha.gmst_upper,shape_est,⋯,dI.rel.pineutral_upper,aic_est,aic_lower,aic_upper,n_est,n_lower,n_upper,alpha.nao_est,alpha.nao_lower,alpha.nao_upper
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ERA5 - GMST only,23.49548,20.05652,27.63639,5.562444,4.041180,7.152993,2.938925,-0.69155290,7.559805,-0.180048937,⋯,43.52453,482.3304,453.5456,500.0504,77,1000,0,NA,NA,NA
ERA5 - GMST + NAO,23.65126,20.33341,27.31903,5.339335,3.875697,6.675930,3.767579,-0.04046242,8.200959,-0.164593578,⋯,57.64815,478.2783,448.1935,494.7081,77,1000,0,-1.548820,-2.772548,-0.1858203
EObs - GMST only (1920-2026),25.78823,22.89643,28.42883,5.759498,4.048542,6.877176,6.410920,3.31135987,9.527385,-0.011965269,⋯,54.81338,663.4172,615.7810,694.2667,107,1000,0,NA,NA,NA
EObs - GMST only (1950-2026),26.23485,22.36150,29.20062,5.994818,3.804557,7.330084,7.131435,2.85837598,11.333839,0.001159175,⋯,65.61162,488.7083,445.2012,515.9884,77,1000,0,NA,NA,NA
EObs - GMST + NAO,26.58263,22.98186,29.92012,5.923338,3.758515,7.164088,8.183790,3.79527065,12.952256,0.006636702,⋯,89.51843,487.3386,444.7727,513.4802,77,1000,0,-1.446595,-3.022974,0.3625181


## Table of results for rp40 event

In [ ]:
varnm <- "rx1day-ondjfm"
rnm <- "s"

res <- read.csv(paste0("res-obs_",varnm,"_",rnm,"_rp40.csv"), row.names = "X")

In [44]:
res

,mu0_est,mu0_lower,mu0_upper,sigma0_est,sigma0_lower,sigma0_upper,alpha.gmst_est,alpha.gmst_lower,alpha.gmst_upper,shape_est,⋯,dI.rel.pineutral_upper,aic_est,aic_lower,aic_upper,n_est,n_lower,n_upper,alpha.nao_est,alpha.nao_lower,alpha.nao_upper
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>
ERA5 - GMST only,23.49548,20.05652,27.63639,5.562444,4.041180,7.152993,2.938925,-0.69155290,7.559805,-0.180048937,⋯,43.52453,482.3304,453.5456,500.0504,77,1000,0,NA,NA,NA
ERA5 - GMST + NAO,23.65126,20.33341,27.31903,5.339335,3.875697,6.675930,3.767579,-0.04046242,8.200959,-0.164593578,⋯,57.64815,478.2783,448.1935,494.7081,77,1000,0,-1.548820,-2.772548,-0.1858203
EObs - GMST only (1920-2026),25.78823,22.89643,28.42883,5.759498,4.048542,6.877176,6.410920,3.31135987,9.527385,-0.011965269,⋯,54.81338,663.4172,615.7810,694.2667,107,1000,0,NA,NA,NA
EObs - GMST only (1950-2026),26.23485,22.36150,29.20062,5.994818,3.804557,7.330084,7.131435,2.85837598,11.333839,0.001159175,⋯,65.61162,488.7083,445.2012,515.9884,77,1000,0,NA,NA,NA
EObs - GMST + NAO,26.58263,22.98186,29.92012,5.923338,3.758515,7.164088,8.183790,3.79527065,12.952256,0.006636702,⋯,89.51843,487.3386,444.7727,513.4802,77,1000,0,-1.446595,-3.022974,0.3625181


In [46]:
cols <- c("event", "return.period", "dI.rel.pi", "dI.rel.naoneutral", "PR.pi", "PR.naoneutral")
tbl <- data.frame(sapply(cols, function(cnm) {
    apply(signif(res[,grepl(cnm, colnames(res)),drop = F], 3), 1, function(r) {
        paste0(r[1], " (",r[2], ", ", r[3], ")")
    })
}))
tbl$event <- gsub(" .+","",tbl$event)
tbl[c(5,2,4,1,3),]

,event,return.period,dI.rel.pi,dI.rel.naoneutral,PR.pi,PR.naoneutral
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
EObs - GMST + NAO,50.8,"40 (14.2, 746)","49.2 (23.6, 77.3)","4.44 (-1.13, 9.33)","48.4 (2.62, Inf)","1.42 (0.929, 4.57)"
ERA5 - GMST + NAO,40.4,"40 (9.02, Inf)","23 (-0.0473, 48)","5.36 (0.841, 9.61)","65.4 (0.983, Inf)","2.1 (1.15, 29.9)"
EObs - GMST only (1950-2026),48.3,"40 (16.4, 297)","42.4 (17.8, 65.6)","0 (0, 0)","29.5 (2.16, Inf)","1 (1, 1)"
ERA5 - GMST only,38.5,"40 (8.13, 17600)","17.7 (-4.12, 43.5)","0 (0, 0)","21.6 (0.402, Inf)","1 (1, 1)"
EObs - GMST only (1920-2026),46.5,"40 (15, 214)","38.2 (20.8, 54.8)","0 (0, 0)","26.3 (3.11, 189000)","1 (1, 1)"
